#  **Web Scraping - Yelp - Construction Valdes**

A partir de la dirección URL de Construction Valdes, hacemos el web scraping de los items que nos interesan.

In [55]:
from lxml import html
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json
from datetime import datetime
from textblob import TextBlob

# Set the option to display all columns
pd.set_option('display.max_columns', None)

# 1. OBTENIENDO TODA LA INFORMACIÓN DE LAS OPINIONES DEL SITIO

base_url = 'https://www.yelp.com/biz/construction-valdes-fresno?start='

# Inicializar listas vacías para almacenar los datos
user_ids = []         # Identificadores de usuario
review_dates = []     # Fechas de las opiniones
user_reviews = []     # Opiniones de los usuarios
ratings_list = []     # Calificaciones

for page_number in range(0, 100, 10):
    # Construir la URL con el número de página actual
    page_url = f'{base_url}{page_number}'
    
    # Enviar una solicitud GET HTTP y recuperar el contenido de la página
    page = requests.get(page_url).text

    # Analizar el contenido HTML utilizando lxml
    tree = html.fromstring(page)

    # Analizar el contenido HTML usando BeautifulSoup para extraer las calificaciones
    soup = BeautifulSoup(page, "html.parser")
    rating_elements = soup.find_all('div', {'class': 'five-stars__09f24__mBKym five-stars--regular__09f24__DgBNj css-1jq1ouh'})
    first_10_rating_elements = rating_elements[3:13]
    ratings_list += [rating['aria-label'].split(' ')[0] for rating in first_10_rating_elements]

    # Bandera para verificar si se cumplen las condiciones
    conditions_met = False

    for i in range(1, 11):
        user_id_xpath = f'/html/body/yelp-react-root/div[1]/div[4]/div/div[1]/div[1]/main/div[4]/div/section/div[2]/ul/li[{i}]/div/div[1]/div/div[1]/div/div/div[2]/div[1]/span/a'
        review_date_xpath = f'/html/body/yelp-react-root/div[1]/div[4]/div/div[1]/div[1]/main/div[4]/div/section/div[2]/ul/li[{i}]/div/div[2]/div/div[2]/span'

        for y in [3, 4]:
            review_xpath = f'/html/body/yelp-react-root/div[1]/div[4]/div/div[1]/div[1]/main/div[4]/div/section/div[2]/ul/li[{i}]/div/div[{y}]/p/span'
            review_element = tree.xpath(review_xpath)
            if review_element:
                user_id_elements = tree.xpath(user_id_xpath)
                review_date_elements = tree.xpath(review_date_xpath)

                # Verificar si se encontraron elementos de ID de usuario y fecha de opinión
                if user_id_elements and review_date_elements:
                    user_id = user_id_elements[0].get('href').split('=')[1]  # Extraer el ID de usuario
                    review_date = review_date_elements[0].text_content()

                    # Convert the review_date string to a datetime object
                    review_date = datetime.strptime(review_date, '%b %d, %Y')

                    user_review = review_element[0].text_content()

                    # Agregar los datos a las listas respectivas
                    user_ids.append(user_id)
                    review_dates.append(review_date)
                    user_reviews.append(user_review)

                    if len(user_ids) == len(ratings_list):
                        conditions_met = True
                        break

    # Verificar las condiciones de finalización
    if len(user_ids) < 10 or not user_ids:
        break


# 2. OBTENIENDO LA INFORMACIÓN DEL NEGOCIO

# Enviar una solicitud GET HTTP y recuperar el contenido de la página
url = 'https://www.yelp.com/biz/construction-valdes-fresno'
page = requests.get(url).text

# Analizar el contenido HTML utilizando lxml
tree = html.fromstring(page)

# Definir las expresiones XPath para acceder a los elementos de nombre, dirección y cantidad de opiniones
name_xpath = '/html/body/yelp-react-root/div[1]/div[4]/div/div[1]/div[1]/main/div[1]/div/div/div[1]/h1'
address_xpath = '/html/body/yelp-react-root/div[1]/div[4]/div/div[1]/div[2]/aside/section[1]/div/div[2]/div/div[1]/p[2]'
reviews_count_xpath = '/html/body/yelp-react-root/div[1]/div[4]/div/div[1]/div[1]/main/div[1]/div/div/div[2]/div[2]/span[2]/a'
stars_business_xpath = '/html/body/yelp-react-root/div[1]/div[4]/div/div[1]/div[1]/main/div[1]/div/div/div[2]/div[2]/span[1]/text()'

# Usar las expresiones XPath para extraer el contenido de los elementos de nombre, dirección y cantidad de opiniones
name_elements = tree.xpath(name_xpath)
address_elements = tree.xpath(address_xpath)
reviews_count_elements = tree.xpath(reviews_count_xpath)
stars_business_elements = tree.xpath(stars_business_xpath)

# Establecer valores iniciales
name = None
address = None
reviews_count = None
stars_business = None

# Verificar si se encontraron resultados para el nombre y la cantidad de opiniones
if name_elements:
    name = name_elements[0].text_content()

if address_elements:
    address = address_elements[0].text_content().strip()

if reviews_count_elements:
    # Extraer la parte numérica del texto y eliminar los caracteres no numéricos
    reviews_count_text = reviews_count_elements[0].text_content()
    reviews_count = re.sub(r'\D', '', reviews_count_text)  # Eliminar caracteres no numéricos

if stars_business_elements:
    stars_business = stars_business_elements[0]

# Datos estáticos de Construction Valdes
business_id = 'qRhlteiXEnElnHhUB81aCw'
city = 'Fresno'
state = 'California'
latitude = '36.7477300'
longitude = '-119.7723700'

# Obtener la fecha actual y darle formato como "Mes Día, Año"
scraping_date = datetime.now().strftime("%Y-%m-%d")

# Inicializar una lista vacía para almacenar los atributos
attributes = []

# Comenzar con la ruta inicial
path_template = '/html/body/yelp-react-root/div[1]/div[4]/div/div[1]/div[1]/main/section[5]/div[2]/div/div/div/div/div/div[2]/span'

# Usar la ruta actual para extraer el atributo
attribute_elements = tree.xpath(path_template)

# Verificar si se encontró un resultado para la ruta actual
if attribute_elements:
    attribute = attribute_elements[0].text_content()
    attributes.append(attribute)
    index += 1

# Inicializar una lista vacía para almacenar las categorías
categories = []

# Comenzar con la ruta inicial para el primer conjunto de categorías
path_template1 = '/html/body/yelp-react-root/div[1]/div[4]/div/div[1]/div[1]/main/section[2]/div[2]/div[1]/div[{}]/a'

# Comenzar con la ruta inicial para el segundo conjunto de categorías
path_template2 = '/html/body/yelp-react-root/div[1]/div[4]/div/div[1]/div[1]/main/section[2]/div[2]/div[2]/div[{}]/a'

# Inicializar el índice para el primer conjunto de categorías
index1 = 1
while True:
    # Construir la ruta completa con el índice actual
    current_path = path_template1.format(index1)

    # Usar la ruta actual para extraer la categoría
    category_elements = tree.xpath(current_path)

    # Verificar si se encontró un resultado para la ruta actual
    if category_elements:
        category_text = category_elements[0].text_content()
        # Eliminar el texto "en X opinión" o "en X opiniones"
        category = category_text.split(' en ')[0]
        categories.append(category)
        index1 += 1
    else:
        # Si no se encuentra ningún resultado, salir del bucle
        break

# Inicializar el índice para el segundo conjunto de categorías
index2 = 1
while True:
    # Construir la ruta completa con el índice actual para el segundo conjunto
    current_path = path_template2.format(index2)

    # Usar la ruta actual para extraer la categoría
    category_elements = tree.xpath(current_path)

    # Verificar si se encontró un resultado para la ruta actual
    if category_elements:
        category_text = category_elements[0].text_content()
        # Eliminar el texto "en X opinión" o "en X opiniones"
        category = category_text.split(' en ')[0]
        categories.append(category)
        index2 += 1
    else:
        # Si no se encuentra ningún resultado, salir del bucle
        break

# Crear un diccionario para almacenar la información del negocio
business_info = {
    'Business Id' : [business_id],
    'Name': [name],
    'City': [city],
    'State' : [state],
    'Latitude' : [latitude],
    'Longitude' : [longitude],
    'Address': [address],
    'Reviews Count': [reviews_count],
    'Stars Business': [stars_business],
    'Attributes': attributes,
    'Categories': categories
}

# Cambiamos el formato de las fechas de los reviews como "Year-Month-Day"
formatted_review_dates = [date.strftime("%Y-%m-%d") for date in review_dates]

# Crear un DataFrame
data = {
    'business_id': [business_info['Business Id'][0] for _ in user_ids],
    'business_name': [business_info['Name'][0] for _ in user_ids],
    'city' : [business_info['City'][0] for _ in user_ids],
    'state': [business_info['State'][0] for _ in user_ids],
    'latitude': [business_info['Latitude'][0] for _ in user_ids],
    'longitude': [business_info['Longitude'][0] for _ in user_ids],
    'stars_business': [business_info['Stars Business'][0] for _ in user_ids],
    'review_count': [business_info['Reviews Count'][0] for _ in user_ids],
    'services': [business_info['Categories'] for _ in user_ids],
    'attributes': [business_info['Attributes'] for _ in user_ids],
    'user_id': user_ids,
    'stars_by_user': ratings_list[:len(user_ids)],
    'review_text': user_reviews,
    'review_date': formatted_review_dates,
    'extract_date': [scraping_date] * len(user_ids)
}

df = pd.DataFrame(data)

#3. Automatic ETL

# Define una función para analizar el sentimiento
def analyze_sentiment(text):
    if isinstance(text, str):
        testimonio = TextBlob(text)
        polarity = testimonio.sentiment.polarity
        if polarity > 0:
            return 'Positiva'
        elif polarity < 0:
            return 'Negativa'
        else:
            return 'Neutral'
    else:
        return 'No es un testimonio válido'

# Aplica el analisis de sentimiento a la columna 'review_text' y crea una columna nueva llamada 'sentiment'
df['sentiment'] = df['review_text'].apply(analyze_sentiment)

# Add an 'index' column
df.insert(0, 'index', range(909331, 909331 + len(df)))

# Add a 'categories' column filled with "Exterior Remodeling or Construction"
df['categories'] = "Exterior Remodeling or Construction"

# Create columns with the names 'biz_category1' to 'biz_category13'
for i in range(1, 14):
    col_name = f'biz_category{i}'
    if i in [2, 13]:
        df[col_name] = True
    else:
        df[col_name] = False

# Drop the 'review_text' column from the DataFrame
df = df.drop(columns=['review_text'])

# Drop the 'services' column from the DataFrame
df = df.drop(columns=['services'])

# Reorder the columns in the DataFrame
df = df[[col for col in df.columns if col != 'extract_date'] + ['extract_date']]

# Imprimir el DataFrame
df

,index,business_id,business_name,city,state,latitude,longitude,stars_business,review_count,attributes,...,biz_category5,biz_category6,biz_category7,biz_category8,biz_category9,biz_category10,biz_category11,biz_category12,biz_category13,extract_date
0,909331,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],...,False,False,False,False,False,False,False,False,True,2023-10-31
1,909332,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],...,False,False,False,False,False,False,False,False,True,2023-10-31
2,909333,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],...,False,False,False,False,False,False,False,False,True,2023-10-31
3,909334,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],...,False,False,False,False,False,False,False,False,True,2023-10-31
4,909335,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],...,False,False,False,False,False,False,False,False,True,2023-10-31
5,909336,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],...,False,False,False,False,False,False,False,False,True,2023-10-31
6,909337,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],...,False,False,False,False,False,False,False,False,True,2023-10-31
7,909338,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],...,False,False,False,False,False,False,False,False,True,2023-10-31
8,909339,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],...,False,False,False,False,False,False,False,False,True,2023-10-31
9,909340,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],...,False,False,False,False,False,False,False,False,True,2023-10-31


In [56]:
# Set the option to display all columns
pd.set_option('display.max_columns', None)

df

,index,business_id,business_name,city,state,latitude,longitude,stars_business,review_count,attributes,user_id,stars_by_user,review_date,sentiment,categories,biz_category1,biz_category2,biz_category3,biz_category4,biz_category5,biz_category6,biz_category7,biz_category8,biz_category9,biz_category10,biz_category11,biz_category12,biz_category13,extract_date
0,909331,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],cQUUrS8yQh1xhz1iXmq8Dg,5,2023-06-28,Positiva,Exterior Remodeling or Construction,False,True,False,False,False,False,False,False,False,False,False,False,True,2023-10-31
1,909332,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],LYfGofx7KeXZcGiZfllT_w,1,2023-10-25,Negativa,Exterior Remodeling or Construction,False,True,False,False,False,False,False,False,False,False,False,False,True,2023-10-31
2,909333,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],QKPsqNFI2i2ji0hVzfoQeA,1,2022-07-15,Positiva,Exterior Remodeling or Construction,False,True,False,False,False,False,False,False,False,False,False,False,True,2023-10-31
3,909334,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],NqLC591iI22DmsOAB9wPnQ,5,2022-08-10,Positiva,Exterior Remodeling or Construction,False,True,False,False,False,False,False,False,False,False,False,False,True,2023-10-31
4,909335,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],sF1asn-G4wIOIos0qH8uhg,5,2022-07-10,Positiva,Exterior Remodeling or Construction,False,True,False,False,False,False,False,False,False,False,False,False,True,2023-10-31
5,909336,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],GcddiZ0iE7e4xFClQTXbRg,5,2022-09-30,Positiva,Exterior Remodeling or Construction,False,True,False,False,False,False,False,False,False,False,False,False,True,2023-10-31
6,909337,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],oFAPNAlF-kzYp5301w5ung,5,2022-10-21,Positiva,Exterior Remodeling or Construction,False,True,False,False,False,False,False,False,False,False,False,False,True,2023-10-31
7,909338,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],RRhpE8OH3Gk4a2035HfRHw,1,2023-01-30,Negativa,Exterior Remodeling or Construction,False,True,False,False,False,False,False,False,False,False,False,False,True,2023-10-31
8,909339,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],iXuEnJxXuT6bJOnWOqFPVg,5,2021-12-31,Positiva,Exterior Remodeling or Construction,False,True,False,False,False,False,False,False,False,False,False,False,True,2023-10-31
9,909340,qRhlteiXEnElnHhUB81aCw,Construction Valdes,Fresno,California,36.7477300,-119.7723700,3.9,14,[Accepts Credit Cards],wbHOzuM3mRubmFjK7d1lhQ,5,2022-08-24,Positiva,Exterior Remodeling or Construction,False,True,False,False,False,False,False,False,False,False,False,False,True,2023-10-31


Guardamos el DataFrame en un archivo CSV para su posterior ETL.

In [58]:
# Obtener la fecha actual en el formato deseado (DDMMAAAA)
current_date = datetime.now().strftime("%d%m%Y")

# Definir el nombre del archivo con la fecha actual
filename = f'cv_scraping_{current_date}.csv'

# Guardar el DataFrame en el archivo
df.to_csv(filename, index=False)

Google Cloud Function:

In [ ]:
import functions_framework
from google.cloud import storage
import pandas as pd
import io
from datetime import datetime  # Import datetime from datetime

@functions_framework.http
def upload_csv_to_gcs(request):
    try:
        # Create a sample dataframe
        data = {
            'Column1': [1, 2, 3],
            'Column2': ['A', 'B', 'C'],
            'Column3': [4.5, 5.6, 6.7]
        }

        df = pd.DataFrame(data)

        # Save the DataFrame to an in-memory CSV file with index=False
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False)

        # Convert the CSV data to bytes
        csv_bytes = csv_buffer.getvalue().encode()

        # Initialize a Google Cloud Storage client
        storage_client = storage.Client()

        # Define your bucket name and the filename
        bucket_name = 'my_bucket_jc'
        current_date = datetime.now().strftime("%d%m%Y")  # Use datetime.now() directly
        file_name = f'sample_{current_date}.csv'

        # Upload the CSV file to your Google Cloud Storage bucket
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(file_name)
        blob.upload_from_string(csv_bytes, content_type='text/csv')

        return f"CSV file '{file_name}' has been uploaded to Google Cloud Storage."

    except Exception as e:
        # Handle exceptions or errors here, e.g., log the error
        return f"An error occurred: {str(e)}"